# முன்னுரிமை உறுப்பினர் மிடில் வேர் கொண்ட ஓட்டல் முன்பதிவு

இந்த நோட்டு புத்தகம் Microsoft Agent Framework பயன்படுத்தி **கருவி அடிப்படையிலான மிடில் வேர்**-ஐ விளக்குகிறது. நாம் நிலைபெறும் ஓட்டல் வேலைநிறைவை விரிவாக்கி, முன்னுரிமை உறுப்பினர்களுக்கு சிறப்பு உரிமைகள் வழங்கும் ஒரு மிடில் வேர் பகுதியை சேர்க்கின்றோம்.

## நீங்கள் என்ன கற்றுக்கொள்ளப்போகிறீர்கள்:
1. **கருவி அடிப்படையிலான மிடில் வேர்**: செயல்பாட்டு முடிவுகளை பிடித்து மாற்றுதல்
2. **சூழல் அணுகல்**: செயல்பாடு முடிந்த பிறகு `context.result` ஐ படித்து மாற்றுதல்
3. **பணியியல் தலையங்க நடைமுறை**: முன்னுரிமை உறுப்பினர் நன்மைகள்
4. **முடிவுகளை மீறல்**: பயனர் நிலையைப் பொருத்து செயல்பாட்டு முடிவுகளை மாற்றுதல்
5. **அதே வேலைநிரல், வேறுபட்ட முடிவுகள்**: மிடில் வேர் தூண்டுதல் நடத்தைகள் மாற்றங்கள்

## மிடில் வேர் உடன் வேலைஞர் கட்டமைப்பு:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## நிலைபெறும் வேலைஞரிலிருந்து முக்கிய வேறுபாடு:

**மிடில் வேர் இல்லாமல்** (14-conditional-workflow.ipynb):
- பாரிஸ் அருகில் அறைகள் இல்லை → alternative_agent க்கு வழிமாற்று

**மிடில் வேர் உடன்** (இந்த நோட்டு புத்தகம்):
- வழக்கமான பயனர் + பாரிஸ் → அறைகள் இல்லை → alternative_agent க்கு வழிமாற்று
- முன்னுரிமை பயனர் + பாரிஸ் → 🌟 மிடில் வேர் மிஞ்சல்! → கிடைக்கும் → booking_agent க்கு வழிமாற்று

## முன் தேவைகள்:
- Microsoft Agent Framework நிறுவப்பட்டிருக்கும்
- நிலைபெறும் வேலைஞர் புரிதல் (14-conditional-workflow.ipynb காண்க)
- GitHub டோக்கன் அல்லது OpenAI API விசை
- மிடில் வேர் முறை உருவாக்கங்கள் அடிப்படைக் கற்றல்


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: கட்டமைக்கப்பட்ட வெளியீடுகளுக்காக பைடான்டிக் மாடல்களை வரையறு

இந்த மாடல்கள் முகவர்கள் திருப்பிபோகும் **வரிசைப்படி**யை வரையறுக்கின்றன. இடைமுகம் கிடைக்கும் முடிவை மாற்றும் போது பின் முன்னுரிமை override என்ற புலத்தை நாங்கள் சேர்த்துள்ளோம்.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: முன்னுரிமை உறுப்பினர்கள் தரவுத்தளத்தை வரையறு

இந்த டெமோவுக்காக, நாம் முன்னுரிமை உறுப்பினர் தரவுத்தளத்தை ஒப்பனை செய்வோம். உற்பத்தியில், இது உண்மையான தரவுத்தளம் அல்லது API ஐ வினவிக்கும்.

**முன்னுரிமை உறுப்பினர்கள்:**
- `alice@example.com` - VIP உறுப்பினர்
- `bob@example.com` - பிரீமியம் உறுப்பினர்  
- `priority_user` - சோதனை கணக்கு


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Step 3: ஹோட்டல் முன்பதிவு கருவியை உருவாக்கவும்

நிபந்தனைச் செயல்முறையைப் போன்றே, ஆனால் இப்போது அது மிடில்‌வேர் மூலம் இடையூறாகப்படும்!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 முன்னுரிமை சரிபார்ப்பு மிடில்வேர் உருவாக்கு (முக்கிய அம்சம்!)

இது இந்த நோட்புக் உள் **முக்கிய செயல்பாடு** ஆகும். மிடில்வேர்:

1. **hostel_booking** செயல்பாட்டை அழைப்பதை தடுக்கிறது  
2. `next(context)` மூலம் செயல்பாட்டை சாதாரணமாக இயற்றி முடிக்கிறது  
3. `context.result` இல் முடிவை ஆய்வு செய்கிறது  
4. பயனர் முன்னுரிமையுடன் மற்றும் அறைகள் இல்லை என்றால் முடிவை மாற்றுகிறது  
5. மாற்றிய முடிவை முகவரிக்கு திருப்பிக் கொடுக்கிறது  

**முக்கிய முன்முறை:**  
```python
async def my_middleware(context, next):
    await next(context)  # செயல்பாட்டை இயக்குக
    # இப்போது context.result செயல்பாட்டின் வெளியீட்டை கொண்டுள்ளது
    if some_condition:
        context.result = new_value  # மீறுக!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## படி 5: வழிசெலுத்தலுக்கு கொள்கை செயல்பாடுகளை வரையறு

கிடைக்கும் வலையமைப்பு வேலைநிரலைப் போன்றவே கொள்கை செயல்பாடுகள் - அவை வழிசெலுத்தலைத் தீர்மானிப்பதற்காக கட்டமைக்கப்பட்ட வெளியீட்டை ஆய்வு செய்கின்றன.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## படி 6: தனிப்பயன் காட்சி இயக்குனரை உருவாக்கவும்

முந்தைய இயக்குனரைப் போலவே - இறுதி வேலைப்பFlowsவில் வெளியீட்டை காட்சி செய்கிறது.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## படி 7: பசுமை மாறிலிகளை ஏற்றவும்

LLM கிளையண்ட் (GitHub மாதிரிகள் அல்லது OpenAI) ஐ கட்டமைக்கவும்.


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## படி 8: மிடில்वेयर உடன் AI முகவர்களை உருவாக்குதல்

**முக்கிய வேறுபாடு:** availability_agent உருவாக்கும்போது, நாம் `middleware` அளவுருவை வழங்குகிறோம்!

இவ்வாறு நாங்கள் priority_check_middleware ஐ முகவரின் செயல்பாட்டு அழைப்பு நெருப்படியில் ஊற்றுகிறோம்.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## படி 9: வேலைநடை கட்டமைக்கவும்

முந்தையதைப் போலவே வேலைநடை கட்டமைப்பு - கிடைக்கும் நிலைப்படி நிபந்தனை வழிசெலுத்தல்.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: சோதனை நிலை 1 - பாரீசில் சாதாரண பயனர் (மீளமைக்காது)

சாதாரண பயனர் பாரீஸ் → அறைகள் இல்லை → alternative_agent-க்கு வழிமாற்று முயல்கிறார்


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: சோதனை வழக்கு 2 - 🌟 பாரிஸ் உள்ள முன்னுரிமை பயனர் (மேலோட்டம் உடன்!)

ஒரு முன்னுரிமை உறுப்பினர் பாரிஸ் க்கு முன்பதிவு செய்ய முயற்சிக்கிறார் → ஆரம்பத்தில் அறைகள் இல்லை → 🌟 மிடில் வெர் மேலோட்டம் செய்கிறது! → booking_agent க்கு வழிதவிர்ப்பு

**இது மிடில்வேர் சக்தியின் முக்கியமான விளக்கம்!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Step 12: Test Case 3 - ஸ்டாக்ஹோல்ம் முன்னுரிமை பயனர் (ஏற்கனவே கிடைக்கின்றது)

முன்னுரிமை பயனர் ஸ்டாக்ஹோல்ம் முயற்சிக்கிறார் → அறைகள் கிடைக்கும் → மேலமைவு தேவையில்லை → booking_agent க்கு பாதைகள்

இதன் மூலம், மிடில்‌வேர் தேவையானபோது மட்டும் செயல்படுகிறது என்பது தெரிகிறது!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## பிரதான சுருக்கங்கள் மற்றும் மிடில்வேர் கருத்துக்கள்

### ✅ நீங்கள் கற்றுக்கொண்டவை:

#### **1. செயல்பாடு அடிப்படையிலான மிடில்வேர் முறை**

மிடில்வேர் ஒரு எளிய அசிங்க் செயல்பாட்டை பயன்படுத்தி செயல்பாட்டு அழைப்புகளை இடைமறிக்கிறது:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # செயல்பாடு நடைமுறைப்படுத்துவதற்கு முன்
    print("Intercepting...")
    
    # செயல்பாட்டை நடத்தியமை
    await next(context)
    
    # செயல்பாடு நடந்து முடிந்த பிறகு - முடிவை பார்க்கவும்
    if context.result:
        # தேவையானால் முடிவை மாற்றவும்
        context.result = modified_value
```

#### **2. சூழல் அணுகல் மற்றும் முடிவை மீறல்**

- `context.function` - அழைக்கப்படும் செயல்பாட்டை அணுகுங்கள்
- `context.arguments` - செயல்பாட்டு அளவுருக்களை படியுங்கள்
- `context.kwargs` - கூடுதல் அளவுருக்களை அணுகுங்கள்
- `await next(context)` - செயல்பாட்டை செயல்படுத்துங்கள்
- `context.result` - செயல்பாட்டின் வெளியீட்டை படியுங்கள்/மாற்றுங்கள்

#### **3. வணிக தந்திரம் செயல்படுத்தல்**

எமது மிடில்வேர் முன்னுரிமை உறுப்பினர் நன்மைகளை செயல்படுத்துகிறது:
- **தனியாரான பயனர்கள்**: மாற்றங்கள் இல்லை, நிலையான வேலைத் தொடர்ச்சி
- **முன்னுரிமை பயனர்கள்**: "சாத்தியமில்லை" என்பதை "கிடைக்கிறது" என மாற்றுதல்
- **நிபந்தனை_logic**: தேவையான போது மட்டுமே மீறல்

#### **4. ஒரே வேலைத் தொடர், வேறுபட்ட முடிவுகள்**

மிடில்வேர் வலிமை:
- ✅ வேலைத் தொடர் அமைப்பில் மாற்றங்கள் இல்லை
- ✅ கருவி செயல்பாட்டில் மாற்றங்கள் இல்லை
- ✅ நிபந்தனை வழிச்செலுத்தலில் மாற்றங்கள் இல்லை
- ✅ மிடில்வேர் மட்டும் → வேறுபட்ட நடத்தைகள்!

### 🚀 இயற்கை உலக பயன்பாடுகள்:

1. **VIP/பிரீமியம் அம்சங்கள்**
   - பிரீமியம் பயனர்களுக்கான விகித வரம்புகளை மீறல்
   - வளங்களுக்கு முன்னுரிமை அணுகல் வழங்கல்
   - பிரீமியம் அம்சங்களை இயக்கத்தில் திறப்பது

2. **A/B சோதனை**
   - பயனர்களை வேறுவிதமான செயல்பாடுகளுக்கு வழிமாற்றுதல்
   - குறிப்பிட்ட பயனர்களுடன் புதிய அம்சங்களை சோதனை செய்தல்
   - படிப்படியாக அம்சங்கள் வெளிப்படுத்தல்

3. **அபாயம் மற்றும் இணக்கம்**
   - செயல்பாட்டு அழைப்புகளை ஆய்வு செய்தல்
   - உணர்வேன் செயல்களை தடுப்பு
   - வணிக விதிகளை கட்டாயப்படுத்தல்

4. **செயல்திறன் மேம்பாடு**
   - குறிப்பிட்ட பயனர்களுக்கான முடிவுகளை சேமித்து வைக்கல்
   - உயிரோட்டமான செயல்களை தவிர்த்தல்
   - விருப்ப மூலங்களை இயக்கல்

5. **பிழைத்திருத்தம் மற்றும் மீண்டும் முயற்சி**
   - பிழைகளை பிடித்து சார்ந்த முறையில் கையாளுதல்
   - மீண்டும் முயற்சி செய்யும் தந்திரம் செயல்படுத்தல்
   - மாற்று செயல்பாடுகளுக்கு மாற்றுதல்

6. **பதிவு மற்றும் கண்காணிப்பு**
   - செயல்பாட்டு நேரங்களை கண்காணித்தல்
   - அளவுருக்கள் மற்றும் முடிவுகளை பதிவு செய்தல்
   - பயன்பாட்டு முறைகளை கண்காணித்தல்

### 🔑 அலங்காரர்களிடமிருந்து முக்கிய வேறுபாடுகள்:

| அம்சம் | அலங்காரர் | மிடில்வேர் |
|---------|-----------|------------|
| **வட்டாரம்** | ஒரே செயல்பாடு | முகவர் உள்ள அனைத்து செயல்பாடுகளும் |
| **நெகிழ்வு** | வரையறுவில் நிலைத்துள்ளது | இயங்கும் நேரத்தில் இயக்குகிறது |
| **சூழல்** | வரையறுக்கப்பட்டது | முழுவட்டார முகவர் சூழல் |
| **கூட்டு அமைப்பு** | பல அலங்காரர்கள் | மிடில்வேர் குழாய் |
| **முகவர்-நுண்ணறிவு** | இல்லை | ஆம் (முகவர் நிலைக்கு அணுகல்) |

### 📚 எப்போது மிடில்வேர் பயன்படுத்துவது:

✅ **மிடில்வேர் பயன்படுத்துகையில்:**
- பயனர்/சேஷன் நிலை படி நடத்தை மாற்ற வேண்டும்
- பல செயல்பாடுகளுக்கு தந்திரம் பொருந்த வேண்டும்
- முகவர் நிலை சூழலுக்கு அணுகல் வேண்டும்
- குறுக்கே செல்லும் கவலைகள் (பதிவு, அங்கீகாரம் மற்றும் பிற)

❌ **மிடில்வேர் பயன்படுத்த கூடாது:**
- எளிய உள்ளீட்டு சரிபார்ப்பு (Pydantic பயன்படுத்தவும்)
- செயல்பாடு சார்ந்த தனிப்பட்ட தந்திரம் (செயல்பாட்டில் வைக்கவும்)
- ஒருமுறை மாற்றங்கள் (செயல்பாட்டிலேயே மாற்றவும்)

### 🎓 மேம்பட்ட முறைமைகள்:

```python
# பல மிடில்‌வேர்கள் (வினைச் சுட்டி வரிசை முக்கியம்!)
middleware=[
    logging_middleware,      # முதலில் பதிவுகளை பதிவு செய்யும்
    auth_middleware,         # அப்பொழுது அங்கீகாரம் பரிசோதிக்கிறது
    cache_middleware,        # அப்பொழுது காட்சே பரிசோதிக்கிறது
    rate_limit_middleware,   # அப்பொழுது விகித வரம்புகளைச் செலுத்துகிறது
    priority_check_middleware  # இறுதியாக முன்னுரிமை பரிசோதனை
]

# நிபந்தனை மிடில்‌வேரின் செயற்பாடு
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # முடிவை மாற்றும்
    else:
        # செயல்பாட்டை முழுமையாக தவிர்த்தல்
        context.result = cached_value
```

### 🔗 தொடர்புடைய கருத்துக்கள்:

- **முகவர் மிடில்வேர்**: agent.run() அழைப்புகளை இடைமறிக்கும்
- **செயல்பாடு மிடில்வேர்**: கருவிய செயல்பாட்டு அழைப்புகளை இடைமறிக்கும் (நாம் பயன்படுத்தியது!)
- **மிடில்வேர் குழாய்**: வரிசைப்படுத்தப்பட்ட மிடில்வேர் சங்கிலி
- **சூழல் பரப்பல்**: மிடில்வேர் சங்கிலியில் நிலையை பரப்புதல்


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**எச்சரிக்கை**:  
இந்தத் தாள் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற ஏ.ஐ. மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாம் துல்லியத்தன்மைக்கு முயற்சி செய்கிறோம் என்றாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்க வாய்ப்பு உள்ளது. அசல் ஆவணம் அதன் தாய்மொழியில் அதிகாரப்பூர்வ மூலமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்காக, தொழில்முறையற்ற மனித மொழிபெயர்ப்பை பரிந்துரைக்கின்றோம். இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்களோர் பொறுப்பேற்க மாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
